# Проектный практикум 3. Учебная задача

## Задача:

Создать нейронную сеть, способную генерировать текстовые отзывы о различных местах на основе определенных входных параметров, таких как категория места, средний рейтинг и ключевые слова.

Данные: https://github.com/yandex/geo-reviews-dataset-2023

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

DF_PATH = Path("/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/geo-reviews-dataset-2023.tskv")

In [6]:
def parsing_line(line):
    # Деление строки по символам табуляции
    parts = line.strip().split("\t")
    entry = {}
    for part in parts:
        # Деление на ключ и значение
        key_value = part.split("=", 1)  # деление по первому знаку =
        if len(key_value) != 2:
            continue  # если нет пары ключ-значение, то строка пропускается
        key, value = key_value
        key = key.strip()
        value = value.strip()

        # Преобразование значение рейтинга float в int
        if key == "rating":  # Замена rating на ключ рейтинга
            try:
                entry[key] = int(float(value))  # Преобразование в float, затем в int
            except ValueError:
                entry[key] = None  # Если получаем ошибку, то сохраняем None
        else:
            entry[key] = value  # С

    return entry

In [11]:
with open(DF_PATH, mode='r') as f:
    data = [parsing_line(line) for line in f if line.strip()]

# Удаление пустых записей (если такие имеются)
data = [entry for entry in data if entry]

# Преобразуем в DataFrame
df = pd.DataFrame(data)

df.sample(5)

,address,name_ru,rating,rubrics,text
28598,"Республика Крым, Симферополь, улица Бела Куна, 43",Грифон,5,АЗС,Качество топлива вполне приемлемое на фоне ста...
420329,"Краснодарский край, Анапа, Красноармейская ули...",Центральный рынок,5,Рынок;Продуктовый рынок,"Я не весь обошла, но мне понравился . Саженцы ..."
115694,"Удмуртская Республика, Ижевск, Удмуртская улиц...",Prof-ton18,5,Тонирование автостёкол;Автостёкла,Отличное место. Большой выбор различных плёнок...
327054,"Свердловская область, Екатеринбург, улица Сакк...",DluxStudio,5,"Салон красоты;Парикмахерская;Визажисты, стилисты","Много лет хожу в это место, все отлично, волос..."
2547,"Красноярск, улица Мечникова, 34",На Мечникова,5,Аптека,Ставлю пятёрку только потому что в этой аптеке...


## Очистка данных

In [12]:
# Проверка наличия и кол-ва пропусков значений
empty = df.isnull().sum()
print(f"Пропуски \n{empty} строк \n")

# Удаление пустых значений, если они есть
if empty.sum() > 0:
  df = df.dropna(subset=['name_ru'])

# Проверка на дубликаты
duplicates = df.duplicated().sum()
print(f"Дубликаты {duplicates} строк")

# Удаление дубликатов если они есть
if duplicates > 0:
  df = df.drop_duplicates(subset=['text']).reset_index(drop=True)

Пропуски 
address      0
name_ru    970
rating       0
rubrics      0
text         0
dtype: int64 строк 

Дубликаты 1 строк


## Подсчёт поличества мест и рубрик

In [13]:
# Подсчёт кол-ва рубрик
rubrics_count = df['rubrics'].nunique()

print(f'Количество уникальных рубрик: {rubrics_count} \n')

# Подсчет кол-ва уникальных адресов
address_count = df['address'].nunique()

print(f'Количество уникальных адресов: {address_count}')

Количество уникальных рубрик: 31722 

Количество уникальных адресов: 191575


## Сохранение полученного DataFrame на Google Drive

In [14]:
df.to_csv("/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/df_prepare.csv", index=False)